In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

import pandas as pd
import pandas_gbq as gbq
import numpy as np
from IPython.core.debugger import set_trace
from tqdm.notebook import trange, tqdm
import json
import os
import re
import datetime
from time import sleep
import pytz
KST = pytz.timezone('Asia/Seoul')

import snscrape
import snscrape.modules.twitter as sntwitter
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [27]:
import pydata_google_auth

credentials = pydata_google_auth.get_user_credentials(
  ['https://www.googleapis.com/auth/bigquery'],
)

In [30]:
from google.cloud import bigquery

bq = bigquery.Client(project='sideb-proejct', credentials=credentials)
dataset = bq.get_dataset('moiber_20210920'); dataset

Dataset(DatasetReference('sideb-proejct', 'moiber_20210920'))

In [31]:
table = bq.get_table('sideb-proejct.moiber_20210920.twitter_feed'); table

Table(TableReference(DatasetReference('sideb-proejct', 'moiber_20210920'), 'twitter_feed'))

In [32]:
table.schema

[SchemaField('id', 'INTEGER', 'REQUIRED', None, (), ()),
 SchemaField('url', 'STRING', 'REQUIRED', None, (), ()),
 SchemaField('created', 'TIMESTAMP', 'REQUIRED', None, (), ()),
 SchemaField('content', 'STRING', 'NULLABLE', None, (), ()),
 SchemaField('userid', 'INTEGER', 'REQUIRED', None, (), ()),
 SchemaField('replyCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('retweetCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('likeCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('quoteCount', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('lang', 'STRING', 'NULLABLE', None, (), ()),
 SchemaField('sourceLabel', 'STRING', 'NULLABLE', None, (), ()),
 SchemaField('media', 'STRING', 'REPEATED', None, (), ()),
 SchemaField('inReplyToUserid', 'INTEGER', 'NULLABLE', None, (), ()),
 SchemaField('mentionedUserids', 'INTEGER', 'REPEATED', None, (), ())]

In [68]:
bq.insert_rows(table, [(
    tw['id'],
    tw['url'],
    tw['date'],
    tw['content'],
    tw['user']['id'],
    tw['replyCount'],
    tw['retweetCount'],
    tw['likeCount'],
    tw['quoteCount'],
    tw['lang'],
    tw['sourceLabel'],
    [_d['fullUrl'] for _d in tw['media']] if tw['media'] else None,
    tw['inReplyToUser']['id'] if tw['inReplyToUser'] else None,
    [_d['id'] for _d in tw['mentionedUsers']] if tw['mentionedUsers'] else None
)])

[]

In [83]:
tw['date'].isoformat()

'2021-08-30T06:32:06+00:00'

In [82]:
tw['date'].astimezone(KST).isoformat()

'2021-08-30T15:32:06+09:00'

In [76]:
def convert2serialize(obj):
    if isinstance(obj, dict):
        return { k: convert2serialize(v) for k, v in obj.items() }
    
    elif hasattr(obj, "_ast"):
        return convert2serialize(obj._ast())
    
    elif not isinstance(obj, str) and hasattr(obj, "__iter__"):
        return [ convert2serialize(v) for v in obj ]
    
    elif hasattr(obj, "__dict__"):
        return {
            k: convert2serialize(v)
            for k, v in obj.__dict__.items()
            if not callable(v) and not k.startswith('_')
        }
    
#     elif isinstance(obj, datetime.date):
#         return obj.astimezone(KST).isoformat()
    
    else:
        return obj

# 테스트 쿼리

In [3]:
n_max = 10
since = '2021-03-01'
until = '2021-08-31'
baseqry = 'goalstudio OR 골스튜디오'
qry = '{baseqry} since:{since} until:{until} lang:ko'.format(baseqry=baseqry, since=since, until=until); qry

'goalstudio OR 골스튜디오 since:2021-03-01 until:2021-08-31 lang:ko'

In [4]:
tweets_list = []

for i, _tw in enumerate(sntwitter.TwitterSearchScraper(qry).get_items()):
    if i > n_max:
        break
        
    tweets_list.append(_tw)
tweets_list

[Tweet(url='https://twitter.com/cloudring0322/status/1432229622075125765', date=datetime.datetime(2021, 8, 30, 6, 32, 6, tzinfo=datetime.timezone.utc), content='나우 갈수록 맘에 안든거 참았는데\n끝나고 나서도 우리한테 빅엿을 날리네\n맨날 똑같은 대본에 똑같은거 연기만 시키고 스튜디오 불편한데만 골라서 가고 방역도 타방송국에 비해 최악임 흔한 가림막도 절대 설치 안해주고 자기들만 마스크 쓰고있음 그래도 마지막 예의는 있는줄 알았는데', renderedContent='나우 갈수록 맘에 안든거 참았는데\n끝나고 나서도 우리한테 빅엿을 날리네\n맨날 똑같은 대본에 똑같은거 연기만 시키고 스튜디오 불편한데만 골라서 가고 방역도 타방송국에 비해 최악임 흔한 가림막도 절대 설치 안해주고 자기들만 마스크 쓰고있음 그래도 마지막 예의는 있는줄 알았는데', id=1432229622075125765, user=User(username='cloudring0322', id=900996263566426112, displayname='구름반지☁️', description='성운이랑 우끼행중🤔 @gooreumseng', rawDescription='성운이랑 우끼행중🤔 @gooreumseng', descriptionUrls=None, verified=False, created=datetime.datetime(2017, 8, 25, 8, 20, 23, tzinfo=datetime.timezone.utc), followersCount=4607, friendsCount=276, statusesCount=122635, favouritesCount=19917, listedCount=113, mediaCount=14587, location='', protected=False, linkUrl=None, linkTcourl=None, profileImageUrl='https:

In [77]:
tw = convert2serialize(tweets_list)[0]; tw

{'url': 'https://twitter.com/cloudring0322/status/1432229622075125765',
 'date': datetime.datetime(2021, 8, 30, 6, 32, 6, tzinfo=datetime.timezone.utc),
 'content': '나우 갈수록 맘에 안든거 참았는데\n끝나고 나서도 우리한테 빅엿을 날리네\n맨날 똑같은 대본에 똑같은거 연기만 시키고 스튜디오 불편한데만 골라서 가고 방역도 타방송국에 비해 최악임 흔한 가림막도 절대 설치 안해주고 자기들만 마스크 쓰고있음 그래도 마지막 예의는 있는줄 알았는데',
 'renderedContent': '나우 갈수록 맘에 안든거 참았는데\n끝나고 나서도 우리한테 빅엿을 날리네\n맨날 똑같은 대본에 똑같은거 연기만 시키고 스튜디오 불편한데만 골라서 가고 방역도 타방송국에 비해 최악임 흔한 가림막도 절대 설치 안해주고 자기들만 마스크 쓰고있음 그래도 마지막 예의는 있는줄 알았는데',
 'id': 1432229622075125765,
 'user': {'username': 'cloudring0322',
  'id': 900996263566426112,
  'displayname': '구름반지☁️',
  'description': '성운이랑 우끼행중🤔 @gooreumseng',
  'rawDescription': '성운이랑 우끼행중🤔 @gooreumseng',
  'descriptionUrls': None,
  'verified': False,
  'created': datetime.datetime(2017, 8, 25, 8, 20, 23, tzinfo=datetime.timezone.utc),
  'followersCount': 4607,
  'friendsCount': 276,
  'statusesCount': 122635,
  'favouritesCount': 19917,
  'listedCount': 113,
  'mediaCount': 14587,
  '

In [5]:
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive',
]

json_file_name = 'sideb-proejct-066e081b7bcd.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1roV5_RPFtJ6z40I6EYGRmqx01Fe6JOOmaJ6nbDrfmkw/edit#gid=2020655845'

# 스프레스시트 문서 가져오기 
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
worksheet = doc.worksheet('20210913')
values = worksheet.get_all_values()
header, rows = values[0], values[1:]

df = pd.DataFrame(rows, columns=header)
df = df[['구분','한글명','영문명 full','영문명 short','트위터 검색쿼리','검색잘됨']].set_index('영문명 short')
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [8]:
qrys = df[df['검색잘됨']=='1']['트위터 검색쿼리'][:148]; qrys

영문명 short
032c                                                       #032c OR #032씨
87mm                                         87엠엠 OR 87앰앰 OR 팔칠앰앰 OR 팔칠엠엠
99percentis                          99percentis OR 99퍼센트이즈 OR 나인티나인퍼센트이즈
jwanderson                           #jwanderson OR "jw앤더슨" OR "제이더블유앤더슨"
lostmanagementcities                                                  엘엠씨
                                              ...                        
medicomtoy                                          medicomtoy OR "메디콤토이"
maisonmargiela          maisonmargiela OR "메종마르지엘라" OR margiela OR "마르...
maisonmined                                        maisonmined OR "메종미네드"
maisonkitsune                                                     "메종키츠네"
modified                                                          "모디파이드"
Name: 트위터 검색쿼리, Length: 120, dtype: object